## Backpropogation using Neural Networks on Iris Data Set, find suitable hidden neurons, learning rate, activation function and performance evaluation

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Function to build and train model
def train_model(hidden_neurons=8, activation='relu', learning_rate=0.01, epochs=100):
    model = Sequential()
    model.add(Dense(hidden_neurons, input_dim=4, activation=activation))
    model.add(Dense(3, activation='softmax'))  # 3 classes in Iris dataset
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0)
    
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=8,
        verbose=0,
        validation_data=(X_test, y_test),
        callbacks=[es]
    )
    
    # Final and best accuracies
    train_acc = history.history['accuracy'][-1]
    val_acc = history.history['val_accuracy'][-1]
    best_val_acc = max(history.history['val_accuracy'])
    
    return {
        "Hidden Neurons": hidden_neurons,
        "Activation": activation,
        "Learning Rate": learning_rate,
        "Train Accuracy": round(train_acc, 4),
        "Validation Accuracy": round(val_acc, 4),
        "Best Validation Accuracy": round(best_val_acc, 4)
    }

# Configurations to try
configs = [
    (4, 'relu', 0.01),
    (8, 'relu', 0.01),
    (16, 'tanh', 0.001),
    (32, 'sigmoid', 0.005),
]

# Run experiments
results = []
for hidden_neurons, activation, lr in configs:
    results.append(train_model(hidden_neurons, activation, lr, epochs=200))

# Display results in a table
df = pd.DataFrame(results)
print(df)


C:\Users\maheit\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


   Hidden Neurons Activation  Learning Rate  Train Accuracy  \
0               4       relu          0.010          0.9833   
1               8       relu          0.010          0.9833   
2              16       tanh          0.001          0.9833   
3              32    sigmoid          0.005          0.9750   

   Validation Accuracy  Best Validation Accuracy  
0                  1.0                       1.0  
1                  1.0                       1.0  
2                  1.0                       1.0  
3                  1.0                       1.0  
